In [ ]:
#-------------------------- Standard Imports --------------------------#
%reload_ext autoreload
%autoreload 2
import os
import kdephys as kde
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import acr
import xarray as xr
from scipy import stats
# ---------------------------- EXTRAS --------------------------------#
from kdephys.plot.main import _title, bp_plot
import kdephys.utils.spectral as sp
bands = sp.bands
from scipy.stats import normaltest
import warnings
warnings.filterwarnings('ignore')
import matplotlib as mpl
from acr.utils import *
pu = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/pub_utils.py', 'pu')
dag = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/data_agg.py', 'dag')
import pingouin as pg
from scipy.stats import shapiro

plt.rcdefaults()
acr.plots.supl()

In [ ]:
plt.rcdefaults()
acr.plots.supl()

In [ ]:
from acr.utils import SOM_BLUE, ACR_BLUE, NNXR_GRAY

MAIN_EXP = 'swisin'
SUBJECT_TYPE = 'som'
MAIN_COLOR = SOM_BLUE
EXP_TYPE_REAL = 'OFFIND'

In [ ]:
subjects, exps = pu.get_subject_list(type=SUBJECT_TYPE, exp=MAIN_EXP)
notebook_figure_root = f'{pu.PAPER_FIGURE_ROOT}/OFF_PERIODS/tonic_comp'
fh_tonic = {}
hd_tonic = {}
fh_offind = {}
hd_offind = {}
for subject, exp in zip(subjects, exps):
    fh_tonic[subject] = acr.io.load_hypno_full_exp(subject, exp)
    hd_tonic[subject] = acr.hypnogram_utils.create_acr_hyp_dict(subject, exp, true_stim=False, extra_rebounds=False)
    fh_offind[subject] = acr.io.load_hypno_full_exp(subject, acr.utils.sub_swi_exps[subject][0])
    hd_offind[subject] = acr.hypnogram_utils.create_acr_hyp_dict(subject, acr.utils.sub_swi_exps[subject][0], true_stim=False, extra_rebounds=False)

In [ ]:
notebook_figure_root = f'{pu.PAPER_FIGURE_ROOT}/OFF_PERIODS/tonic_comp'

# Get the basic DataFrames

In [ ]:
from acr.utils import pub_data_root
oodfs = []
for f in os.listdir(f'{pub_data_root}/full_context_dfs_tonic_off_comparisons'):
    if f.endswith('.parquet'):
        oodfs.append(pl.read_parquet(f'{pub_data_root}/full_context_dfs_tonic_off_comparisons/{f}'))
oodf = pl.concat(oodfs)

In [ ]:
oodf_tonic = oodf.filter(pl.col('exp')=='swisin')
oodf_tonic = oodf_tonic.with_columns(xtype=pl.lit('tonic'))
oodf_offind = oodf.filter(pl.col('exp')!='swisin')
oodf_offind = oodf_offind.with_columns(xtype=pl.lit('offind'))

In [ ]:
cc2r = ['sw_peak', 'pre_off_mua_count', 'post_off_mua_count', 'intra_off_mua_count', 'span_avg']
method = 'median'
ref_to_col = 'full_bl'
ref_to_val = 'True'
for col in cc2r:
    subdfs = []
    for subject in subjects:
        subdf = oodf_tonic.filter(pl.col('subject')==subject)
        subdf = acr.oo_utils.relativize_oodf(subdf, ref_to_col=ref_to_col, ref_to_val=ref_to_val, avg_method=method, col_to_relativize=col)
        subdfs.append(subdf)
    oodf_tonic = pl.concat(subdfs)

    subdfs = []
    for subject in subjects:
        subdf = oodf_offind.filter(pl.col('subject')==subject)
        subdf = acr.oo_utils.relativize_oodf(subdf, ref_to_col=ref_to_col, ref_to_val=ref_to_val, avg_method=method, col_to_relativize=col)
        subdfs.append(subdf)
    oodf_offind = pl.concat(subdfs)
oodf = pl.concat([oodf_tonic, oodf_offind])

In [ ]:
# reduce some of the parietal-frontal variance here by filtering out some of the random OFF periods that are more likely during the waking conditions due to the increased MUA sparsity on the frontal probes.
# we can do this simple filtering by removing OFF periods that are not associated with some positive deflection of the layer 5 LFP and some minimal spiking activity during the preceding ON period
# This applies to both probes in all conditions, but only to parietal subjects, as the frontal subjects are already just much less likely to get some of the "random/statistical" OFF periods, so even very 
# minor differences in this rate can cause large variance between the frontal vs parietal subjects, which is not really the thing we want to look at here. 
frontal_subs = ['ACR_25', 'ACR_39']
oodf_offind = oodf_offind.filter(
    (((pl.col('sw_peak_rel')>1)&(pl.col('pre_off_mua_count_rel')>.33)) | (pl.col('subject').is_in(frontal_subs)) | ~(pl.col('condition').is_in(['stim', 'early_sd', 'late_sd'])))
)
frontal_subs = ['ACR_25', 'ACR_39']
oodf_tonic = oodf_tonic.filter(
    (((pl.col('sw_peak_rel')>1)&(pl.col('pre_off_mua_count_rel')>.33)) | (pl.col('subject').is_in(frontal_subs)) | ~(pl.col('condition').is_in(['stim', 'early_sd', 'late_sd'])))
)

# Frequency During Stim

In [ ]:
ooton = oodf_tonic.filter()
ooind = oodf_offind.filter()

In [ ]:
hdf_freqs = []
ref_to_col = 'full_bl'
ref_to_val = 'True'
avg_method = 'mean'
col_to_relativize = 'off_freq_per_s'

for subject in subjects:
    subdf = ooton.filter(pl.col('subject')==subject)
    subfreq = acr.oo_utils.calc_off_freq(subdf, window='5s')
    subfreq = acr.oo_utils.label_oodf_with_states(subfreq, fh_tonic[subject])
    subfreq = acr.oo_utils.label_oodf_full_bl(subfreq)
    subfreq = acr.oo_utils.label_oodf_hyp_dict_conditions(subfreq, hd_tonic[subject])    
    subfreq = acr.oo_utils.relativize_oodf(subfreq, ref_to_col=ref_to_col, ref_to_val=ref_to_val, avg_method=avg_method, col_to_relativize=col_to_relativize) 
    hdf_freqs.append(subfreq)
freq_tonic = pl.concat(hdf_freqs)
freq_tonic = freq_tonic.with_columns(xtype=pl.lit('tonic'))

In [ ]:
hdf_freqs = []

for subject in subjects:
    subdf = ooind.filter(pl.col('subject')==subject)
    subfreq = acr.oo_utils.calc_off_freq(subdf, window='5s')
    subfreq = acr.oo_utils.label_oodf_with_states(subfreq, fh_offind[subject])
    subfreq = acr.oo_utils.label_oodf_full_bl(subfreq)
    subfreq = acr.oo_utils.label_oodf_hyp_dict_conditions(subfreq, hd_offind[subject])    
    subfreq = acr.oo_utils.relativize_oodf(subfreq, ref_to_col=ref_to_col, ref_to_val=ref_to_val, avg_method=avg_method, col_to_relativize=col_to_relativize) 
    hdf_freqs.append(subfreq)
freq_offind = pl.concat(hdf_freqs)
freq_offind = freq_offind.with_columns(xtype=pl.lit('offind'))

In [ ]:
fig_id = 'off_frequency_during_stim--TONIC'
fig_name = f'{SUBJECT_TYPE}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

fr = pl.concat([freq_tonic, freq_offind])
fmean = fr.group_by(['xtype', 'subject', 'probe', 'condition']).agg(pl.col('off_freq_per_s_rel').mean()).sort('xtype', 'subject', 'probe', 'condition')
nnxo_ton = fmean.cdn('stim').xt('tonic').prb('NNXo')['off_freq_per_s_rel'].to_numpy()
nnxr_ton = fmean.cdn('stim').xt('tonic').prb('NNXr')['off_freq_per_s_rel'].to_numpy()
ton_rel = nnxo_ton/nnxr_ton
f, ax = acr.plots.gen_paired_boxplot(nnxr_ton, nnxo_ton, alphas=[0.9, 0.6])
ax.set_xticklabels(['Contra. Control', 'Optrode'])
ax.set_ylim(-0.02, 0.3)
f.savefig(fig_path, dpi=600, transparent=True, bbox_inches='tight')

In [ ]:
# =============================
# ========== STATS ============
# =============================
write = True

nnxr = nnxr_ton
nnxo = nnxo_ton
diffs = nnxo-nnxr
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
# stats = pg.wilcoxon(ton_rel, offind_rel)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(ton_rel))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    #acr.stats.write_stats_result(stats_name, 'wilcoxon', stats['W-val'][0], stats['p-val'][0], 'r, RBC', [r, stats['RBC'][0]])
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'optrode': nnxo, 'subject': np.arange(len(nnxo))})
    pu.write_source_data(source_data, stats_name)
stats

In [ ]:
fig_id = 'off_frequency_during_stim--OFFIND'
fig_name = f'{SUBJECT_TYPE}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

nnxo_offind = fmean.cdn('stim').xt('offind').prb('NNXo')['off_freq_per_s_rel'].to_numpy()
nnxr_offind = fmean.cdn('stim').xt('offind').prb('NNXr')['off_freq_per_s_rel'].to_numpy()
offind_rel = nnxo_offind/nnxr_offind
f, ax = acr.plots.gen_paired_boxplot(nnxr_offind, nnxo_offind, alphas=[0.9, 0.9])
ax.set_xticklabels(['Contra. Control', 'Optrode'])
f.savefig(fig_path, dpi=600, transparent=True, bbox_inches='tight')

In [ ]:
# =============================
# ========== STATS ============
# =============================
write = True

nnxr = nnxr_offind
nnxo = nnxo_offind
diffs = nnxo-nnxr
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
# stats = pg.wilcoxon(ton_rel, offind_rel)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(ton_rel))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    #acr.stats.write_stats_result(stats_name, 'wilcoxon', stats['W-val'][0], stats['p-val'][0], 'r, RBC', [r, stats['RBC'][0]])
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'optrode': nnxo, 'subject': np.arange(len(nnxo))})
    pu.write_source_data(source_data, stats_name)
stats

In [ ]:
fig_id = 'off_frequency_during_stim--TON_VS_OFFIND_rel2CC'
fig_name = f'{SUBJECT_TYPE}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

f, ax = acr.plots.gen_paired_boxplot(ton_rel, offind_rel, alphas=[0.6, 0.9])
ax.set_xticklabels(['Tonic', 'OFF Induction'])
f.savefig(fig_path, dpi=600, transparent=True, bbox_inches='tight')

In [ ]:
# =============================
# ========== STATS ============
# =============================
write = True

nnxr = ton_rel
nnxo = offind_rel
diffs = nnxo-nnxr
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
# stats = pg.wilcoxon(ton_rel, offind_rel)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(ton_rel))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    #acr.stats.write_stats_result(stats_name, 'wilcoxon', stats['W-val'][0], stats['p-val'][0], 'r, RBC', [r, stats['RBC'][0]])
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'tonic': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxo))})
    pu.write_source_data(source_data, stats_name)
stats

# RAW checks

In [ ]:
oomean = oodf.group_by(['xtype', 'subject', 'probe', 'condition']).agg(pl.col('sw_peak_rel').mean()).sort('xtype', 'subject', 'probe', 'condition')

nnxo = oomean.cdn('stim').xt('tonic').prb('NNXo')['sw_peak_rel'].to_numpy()
nnxr = oomean.cdn('stim').xt('tonic').prb('NNXr')['sw_peak_rel'].to_numpy()
ton_rel = nnxo/nnxr
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo)
ax.set_title('Tonic, sw_peak_rel')
plt.show()

nnxo = oomean.cdn('stim').xt('offind').prb('NNXo')['sw_peak_rel'].to_numpy()
nnxr = oomean.cdn('stim').xt('offind').prb('NNXr')['sw_peak_rel'].to_numpy()
offind_rel = nnxo/nnxr
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo)
ax.set_title('Off-induction, sw_peak_rel')
plt.show()

f, ax = acr.plots.gen_paired_boxplot(ton_rel, offind_rel, colors=['dodgerblue', SOM_BLUE])
ax.set_title('Tonic vs. Off-induction, sw_peak_rel')
plt.show()

In [ ]:
oomean = oodf.group_by(['xtype', 'subject', 'probe', 'condition']).agg(pl.col('pre_off_mua_count_rel').mean()).sort('xtype', 'subject', 'probe', 'condition')

nnxo = oomean.cdn('stim').xt('tonic').prb('NNXo')['pre_off_mua_count_rel'].to_numpy()
nnxr = oomean.cdn('stim').xt('tonic').prb('NNXr')['pre_off_mua_count_rel'].to_numpy()
ton_rel = nnxo/nnxr
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo)
ax.set_title('Tonic, pre_off_mua_count_rel')
plt.show()

nnxo = oomean.cdn('stim').xt('offind').prb('NNXo')['pre_off_mua_count_rel'].to_numpy()
nnxr = oomean.cdn('stim').xt('offind').prb('NNXr')['pre_off_mua_count_rel'].to_numpy()
offind_rel = nnxo/nnxr
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo)
ax.set_title('Off-induction, pre_off_mua_count_rel')
plt.show()

f, ax = acr.plots.gen_paired_boxplot(ton_rel, offind_rel, colors=['dodgerblue', SOM_BLUE])
ax.set_title('Tonic vs. Off-induction, pre_off_mua_count_rel')
plt.show()

In [ ]:
oomean = oodf.group_by(['xtype', 'subject', 'probe', 'condition']).agg(pl.col('pre_off_mua_count_rel').mean()).sort('xtype', 'subject', 'probe', 'condition')

nnxo = oomean.cdn('stim').xt('tonic').prb('NNXo')['pre_off_mua_count_rel'].to_numpy()
nnxr = oomean.cdn('stim').xt('tonic').prb('NNXr')['pre_off_mua_count_rel'].to_numpy()
ton_rel = nnxo/nnxr
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo)
ax.set_title('Tonic, pre_off_mua_count_rel')
plt.show()

nnxo = oomean.cdn('stim').xt('offind').prb('NNXo')['pre_off_mua_count_rel'].to_numpy()
nnxr = oomean.cdn('stim').xt('offind').prb('NNXr')['pre_off_mua_count_rel'].to_numpy()
offind_rel = nnxo/nnxr
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo)
ax.set_title('Off-induction, pre_off_mua_count_rel')
plt.show()

f, ax = acr.plots.gen_paired_boxplot(ton_rel, offind_rel, colors=['dodgerblue', SOM_BLUE])
ax.set_title('Tonic vs. Off-induction, pre_off_mua_count_rel')
plt.show()

In [ ]:
oomean = oodf.group_by(['xtype', 'subject', 'probe', 'condition']).agg(pl.col('post_off_mua_count_rel').mean()).sort('xtype', 'subject', 'probe', 'condition')

nnxo = oomean.cdn('stim').xt('tonic').prb('NNXo')['post_off_mua_count_rel'].to_numpy()
nnxr = oomean.cdn('stim').xt('tonic').prb('NNXr')['post_off_mua_count_rel'].to_numpy()
ton_rel = nnxo/nnxr
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo)
ax.set_title('Tonic, post_off_mua_count_rel')
plt.show()

nnxo = oomean.cdn('stim').xt('offind').prb('NNXo')['post_off_mua_count_rel'].to_numpy()
nnxr = oomean.cdn('stim').xt('offind').prb('NNXr')['post_off_mua_count_rel'].to_numpy()
offind_rel = nnxo/nnxr
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo)
ax.set_title('Off-induction, post_off_mua_count_rel')
plt.show()

f, ax = acr.plots.gen_paired_boxplot(ton_rel, offind_rel, colors=['dodgerblue', SOM_BLUE])
ax.set_title('Tonic vs. Off-induction, post_off_mua_count_rel')
plt.show()

In [ ]:
oomean = oodf.group_by(['xtype', 'subject', 'probe', 'condition']).agg(pl.col('intra_off_mua_count_rel').mean()).sort('xtype', 'subject', 'probe', 'condition')

nnxo = oomean.cdn('stim').xt('tonic').prb('NNXo')['intra_off_mua_count_rel'].to_numpy()
nnxr = oomean.cdn('stim').xt('tonic').prb('NNXr')['intra_off_mua_count_rel'].to_numpy()
ton_rel = nnxo/nnxr
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo)
ax.set_title('Tonic, intra_off_mua_count_rel')
plt.show()

nnxo = oomean.cdn('stim').xt('offind').prb('NNXo')['intra_off_mua_count_rel'].to_numpy()
nnxr = oomean.cdn('stim').xt('offind').prb('NNXr')['intra_off_mua_count_rel'].to_numpy()
offind_rel = nnxo/nnxr
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo)
ax.set_title('Off-induction, intra_off_mua_count_rel')
plt.show()

f, ax = acr.plots.gen_paired_boxplot(ton_rel, offind_rel, colors=['dodgerblue', SOM_BLUE])
ax.set_title('Tonic vs. Off-induction, intra_off_mua_count_rel')
plt.show()

In [ ]:
oomean = oodf.group_by(['xtype', 'subject', 'probe', 'condition']).agg(pl.col('span_avg_rel').mean()).sort('xtype', 'subject', 'probe', 'condition')

nnxo = oomean.cdn('stim').xt('tonic').prb('NNXo')['span_avg_rel'].to_numpy()
nnxr = oomean.cdn('stim').xt('tonic').prb('NNXr')['span_avg_rel'].to_numpy()
ton_rel = nnxo/nnxr
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo)
ax.set_title('Tonic, span_avg_rel')
plt.show()

nnxo = oomean.cdn('stim').xt('offind').prb('NNXo')['span_avg_rel'].to_numpy()
nnxr = oomean.cdn('stim').xt('offind').prb('NNXr')['span_avg_rel'].to_numpy()
offind_rel = nnxo/nnxr
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo)
ax.set_title('Off-induction, span_avg_rel')
plt.show()

f, ax = acr.plots.gen_paired_boxplot(ton_rel, offind_rel, colors=['dodgerblue', SOM_BLUE])
ax.set_title('Tonic vs. Off-induction, span_avg_rel')
plt.show()

In [ ]:
oomean = oodf.group_by(['xtype', 'subject', 'probe', 'condition']).agg(pl.col('duration_rel').mean()).sort('xtype', 'subject', 'probe', 'condition')

nnxo = oomean.cdn('stim').xt('tonic').prb('NNXo')['duration_rel'].to_numpy()
nnxr = oomean.cdn('stim').xt('tonic').prb('NNXr')['duration_rel'].to_numpy()
ton_rel = nnxo/nnxr
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo)
ax.set_title('Tonic, duration_rel')
plt.show()

nnxo = oomean.cdn('stim').xt('offind').prb('NNXo')['duration_rel'].to_numpy()
nnxr = oomean.cdn('stim').xt('offind').prb('NNXr')['duration_rel'].to_numpy()
offind_rel = nnxo/nnxr
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo)
ax.set_title('Off-induction, duration_rel')
plt.show()

f, ax = acr.plots.gen_paired_boxplot(ton_rel, offind_rel, colors=['dodgerblue', SOM_BLUE])
ax.set_title('Tonic vs. Off-induction, duration_rel')
plt.show()